### Boilerplate

In [159]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [160]:
from __future__ import division
from bs4 import BeautifulSoup
from collections import OrderedDict
from datetime import date
from datetime import datetime
import json
import math
import os
from pandas import *
import requests
from selenium import webdriver
from time import sleep
import wikipedia

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

options.display.max_rows = 500

### Sourcing data
* identify data sources
* scrape data
    * [x] Injuries: URL
    * [x] Weekly rank: URL
    * [x] Historical lines/scores: URL
    * [x] Play-by-play?: (needs to be parsed) but backup link: URL
    * [x] Historical weather: used .. but backup link: URL
    * [x] Snap counts (could be helpful with injuries): URL
    * [x] Head coach info (# wins, winning %, age)
    * [x] Weekly stats -- player/team: URL
    * [x] division/conference
    * [x] travel data between teams, or timezone of location (timezone change and direction of change)
    * [x] timezones
    * NFL API: URL
    * __sports info solutions__

#### TODO: convert to .py files, that (1) do original data load, and (2) find only net new data

In [49]:
DATA_DIR = '../../football_data/'

### Team ID Mapping for site:

In [5]:
## code after having BeautifulSoup object
pfr_teams = {}
for tbl in soup.findAll('table', {'id': 'teams_active'}):
    tbl = tbl.findAll('tbody')[0]
    for row in tbl.findAll('tr'):
        for team in row.findAll('th'):
            for a in team.findAll('a'):
                pfr_teams[a.attrs['href'].split('/')[2]] = a.text

In [195]:
pfr_teams

{'atl': u'Atlanta Falcons',
 'buf': u'Buffalo Bills',
 'car': u'Carolina Panthers',
 'chi': u'Chicago Bears',
 'cin': u'Cincinnati Bengals',
 'cle': u'Cleveland Browns',
 'clt': u'Indianapolis Colts',
 'crd': u'Arizona Cardinals',
 'dal': u'Dallas Cowboys',
 'den': u'Denver Broncos',
 'det': u'Detroit Lions',
 'gnb': u'Green Bay Packers',
 'htx': u'Houston Texans',
 'jax': u'Jacksonville Jaguars',
 'kan': u'Kansas City Chiefs',
 'mia': u'Miami Dolphins',
 'min': u'Minnesota Vikings',
 'nor': u'New Orleans Saints',
 'nwe': u'New England Patriots',
 'nyg': u'New York Giants',
 'nyj': u'New York Jets',
 'oti': u'Tennessee Titans',
 'phi': u'Philadelphia Eagles',
 'pit': u'Pittsburgh Steelers',
 'rai': u'Oakland Raiders',
 'ram': u'Los Angeles Rams',
 'rav': u'Baltimore Ravens',
 'sdg': u'Los Angeles Chargers',
 'sea': u'Seattle Seahawks',
 'sfo': u'San Francisco 49ers',
 'tam': u'Tampa Bay Buccaneers',
 'was': u'Washington Redskins'}

### Injuries

In [196]:
def get_injury_team_year(team, year, driver):
    '''scrape for injury info:
    '''
    i = 0
    ## code after having BeautifulSoup object
    tbl = soup.findAll('div', {'id':'div_team_injuries'})[0]
    tbl = tbl.findAll('tbody')[0]

    def get_status(col):
        status = col.attrs['class'][2:]
        if len(status) == 0:
            return ['A', False]
        elif len(status) == 1:
            return [status[0], False]
        else:
            return [status[0], True]

    def get_injury(col):
        if 'data-tip' in col.attrs.keys():
            return col.attrs['data-tip'].split(': ')[-1]
        else:
            return ''

    players = {}
    for row in tbl.findAll('tr'):
        for th in row.findAll('th'):
            if 'data-append-csv' not in th.attrs.keys():
                return None
            pid = th.attrs['data-append-csv']
            name = th.text
        for col in row.findAll('td'):
            status, dnp = get_status(col)

            week = int(col.attrs['data-stat'].split('_')[-1])
            inj = get_injury(col)
            players[i] = {
                'pid': pid,
                'status': status, 
                'dnp': dnp, 
                'inj': inj,
                'name': name,
                'team': team,
                'week': week, 
                'year': year
            }
            i += 1

    return DataFrame.from_dict(players, orient='index')

In [201]:
first = False
for team in pfr_teams.keys():
    for year in np.arange(2018,2019):
        df = get_injury_team_year(team, year, driver)
        if df is not None:
            if first is True:
                injuries = df
                first = False
            else:
                injuries = injuries.append(df).reset_index(drop=True)

In [202]:
print injuries.shape
injuries.head()

(167208, 8)


,status,week,dnp,inj,team,year,pid,name
0,A,1,False,,Los Angeles Rams,2011,AhYoC.00,C.J. Ah You
1,out,2,True,wrist,Los Angeles Rams,2011,AhYoC.00,C.J. Ah You
2,questionable,3,True,wrist,Los Angeles Rams,2011,AhYoC.00,C.J. Ah You
3,A,4,False,,Los Angeles Rams,2011,AhYoC.00,C.J. Ah You
4,A,6,False,,Los Angeles Rams,2011,AhYoC.00,C.J. Ah You


In [203]:
injuries['team'] = injuries['team'].map(pfr_teams)

In [204]:
injuries.to_csv('all_injuries.csv'.format(DATA_DIR))

### Historical Lines:
* Site has a horrible data format. lines and O/U are inconsistent
 * ML will indicate who is the favorite (open or close?)
 * lower of 2H is the spread; higher is the O/U
 * open/close have 4 combos. take max open and max close

In [6]:
df = read_excel('URL')\
        .reset_index(drop=True)
df.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,906,451,V,Atlanta,3,3,0,6,12,46.5,pk,-119,pk
1,906,452,H,Philadelphia,0,3,7,8,18,4,44,-101,21
2,909,453,V,Pittsburgh,0,7,14,0,21,6.5,3.5,-185,pk
3,909,454,H,Cleveland,0,0,7,14,21,47,41,155,20
4,909,455,V,SanFrancisco,0,3,10,3,16,47,46,230,21.5


#### Check that it alternates Visitor-Home

In [8]:
df = df.reset_index(drop=False)
df['exp'] = df['index'].apply(lambda x: 'V' if x % 2 == 0 else 'H')
df[df['exp'] != df['VH']] # neutral location

,index,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,exp


In [15]:
def process_lines(df, season):
    '''takes in a season of lines. pre-processes the funky schema/pattern
    and returns a df ready for analysis'''
    df['game_id'] = map(lambda x: np.floor(x/2.), np.arange(df.shape[0]))

    def clean_pk(x):
        '''takes in a value that is either a numeric or string.
        if string, it is a pickem (value==pk). return 0. otherwise,
        return original value'''
        try:
            return float(x)
        except:
            return 0.

    games_dict = {}
    for idx in df.index:
        ## visitor
        if idx % 2 == 0:
            game_id = df.loc[idx, 'game_id']
            games_dict[game_id] = {
                'Date': df.loc[idx, 'Date'],
                'V_Team': df.loc[idx, 'Team'],
                'V_Q1': df.loc[idx, '1st'],
                'V_Q2': df.loc[idx, '2nd'],
                'V_Q3': df.loc[idx, '3rd'],
                'V_Q4': df.loc[idx, '4th'],
                'V_Final': df.loc[idx, 'Final'],
                'v_open': clean_pk(df.loc[idx, 'Open']),
                'v_close': clean_pk(df.loc[idx, 'Close']),
                'V_ML': df.loc[idx, 'ML'],
                'v_2h': df.loc[idx, '2H'],
                'Neutral': df.loc[idx, 'VH'] == 'N'
            }

        ## home
        else:
            games_dict[game_id]['H_Team'] = df.loc[idx, 'Team']
            games_dict[game_id]['H_Q1'] = df.loc[idx, '1st']
            games_dict[game_id]['H_Q2'] = df.loc[idx, '2nd']
            games_dict[game_id]['H_Q3'] = df.loc[idx, '3rd']
            games_dict[game_id]['H_Q4'] = df.loc[idx, '4th']
            games_dict[game_id]['H_Final'] = df.loc[idx, 'Final']
            games_dict[game_id]['h_open'] = clean_pk(df.loc[idx, 'Open'])
            games_dict[game_id]['h_close'] = clean_pk(df.loc[idx, 'Close'])
            games_dict[game_id]['H_ML'] = df.loc[idx, 'ML']
            games_dict[game_id]['h_2h'] = df.loc[idx, '2H']

    def ml_fav(x):
        '''whoever has smaller ML
        is the ML fav'''
        h_ml, v_ml = x
        if h_ml == v_ml:
            return 'pickem'
        elif h_ml < v_ml:
            return 'H'
        else:
            return 'V'

    def spread_fav(home, vis):
        '''the smaller number between home/visitor for both
        open and close is the favorite. 
        this returns the spread, favorite, or O/U'''
        fav = 'H' if home < vis else 'V'
        if home < vis:
            fav = 'H'
            ou = vis
            spread = home
        else:
            fav = 'V'
            ou = home
            spread = vis

        return [fav, ou, spread]

    games = DataFrame.from_dict(games_dict, orient='index')
    # ML will indicate who is the favorite (open or close?)
    games['ML_Fav'] = games[['H_ML','V_ML']].apply(ml_fav, axis=1)
    # favorite is the one with the lower (spread)
    for bet_type, bet_name in [('open','Open'),('close','Close'),('2h','2H')]:
        info_col = '{}_info'.format(bet_type)
        games[info_col] = games[
                    ['h_{}'.format(bet_type),
                     'v_{}'.format(bet_type)]
                ].apply(
                    lambda (home,vis): spread_fav(home, vis), axis=1
                )

        for i, (field, field_name) in enumerate([
                                        ('fav','Fav'),
                                        ('ou','OU'),
                                        ('spread','Spread')]):
            data_field = '{}_{}'.format(bet_name, field_name)
            games[data_field] = games[info_col].apply(lambda x: x[i])

    games['Season'] = season
    games = games[[
        'Season','Date','H_Team','V_Team','H_Final','V_Final','H_ML','V_ML','ML_Fav',
        'Open_Fav','Open_OU','Open_Spread','Close_Fav','Close_OU','Close_Spread',
        '2H_Fav','2H_OU','2H_Spread','H_Q1','H_Q2','H_Q3','H_Q4',
        'V_Q1','V_Q2','V_Q3','V_Q4'
    ]]

    assert games[games['Open_Spread'] >= games['Open_OU']].shape[0] == 0
    assert games[games['Close_Spread'] >= games['Close_OU']].shape[0] == 0
    assert games[games['2H_Spread'] >= games['2H_OU']].shape[0] == 0

    return games

In [16]:
dfs = []
for season in np.arange(2019, 2007, -1):
    print season
    df = read_excel(
        'URL' + 
        '{}-{}.xlsx'.format(season-1, str(season)[-2:]))\
            .reset_index(drop=True)
    dfs.append(process_lines(df, season-1))

2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008


In [17]:
all_lines = reduce(lambda x,y: x.append(y).reset_index(drop=True), dfs)
all_lines.shape

(3160, 26)

In [19]:
all_lines['Season'].value_counts().sort_index()

2007    267
2008    267
2009    267
2010    267
2011    267
2012    267
2013    267
2014    267
2015    267
2016    267
2017    267
2018    223
Name: Season, dtype: int64

#### Fix erroneous game dates for historical lines

In [20]:
## These games say 10-17 but were actually on 10-18
## caught this when exploring data
idx = all_lines[
    (all_lines['Season'] == 2015)
    & (all_lines['Date'] == 1017)
].index
all_lines.loc[idx, 'Date'] = 1018

## 1224 --> 1225, and the rest are all similar
idx = all_lines[
    (all_lines['Season'] == 2010)
    & (all_lines['Date'] == 1224)
    & (all_lines['H_Team'] == 'Arizona')
].index
all_lines.loc[idx, 'Date'] = 1225

idx = all_lines[
    (all_lines['Season'] == 2014)
    & (all_lines['Date'] == 930)
    & (all_lines['H_Team'] == 'KansasCity')
].index
all_lines.loc[idx, 'Date'] = 929

idx = all_lines[
    (all_lines['Season'] == 2015)
    & (all_lines['Date'] == 123)
    & (all_lines['H_Team'] == 'Denver')
].index
all_lines.loc[idx, 'Date'] = 124

idx = all_lines[
    (all_lines['Season'] == 2015)
    & (all_lines['Date'] == 123)
    & (all_lines['H_Team'] == 'Carolina')
].index
all_lines.loc[idx, 'Date'] = 124

In [21]:
from datetime import date
def map_dates(x):
    dt, yy = x
    dt = str(dt)
    dd = int(dt[-2:])
    mm = int(dt[:-2])
    yy = int(yy)
    
    if mm < 6:
        return date(yy+1, mm, dd)
    else:
        return date(yy, mm, dd)
    
all_lines['Date'] = all_lines[['Date','Season']].apply(map_dates, axis=1)

In [26]:
all_lines.to_csv('{}all_lines.csv'.format(DATA_DIR))

### Alternative For Lines: 

In [14]:
started = False
dfs = []
for years in [(2018, 2014), (2014, 2010), (2010, 2006), (2006, 2000)]:
    curr = read_csv('URL')
    dfs.append(curr)

In [15]:
lines = reduce(
    lambda x,y: x.append(y[x.columns]).reset_index(drop=True),
    dfs
)

In [16]:
lines.head()

,Date,Vis Team,Home Team,Vis Spread,Home Spread,Over Under
0,2014-09-04,Packers,Seahawks,5.0,-5.0,47.0
1,2014-09-07,Saints,Falcons,-3.0,3.0,50.5
2,2014-09-07,Vikings,Rams,3.0,-3.0,43.5
3,2014-09-07,Browns,Steelers,6.0,-6.0,41.5
4,2014-09-07,Jaguars,Eagles,10.0,-10.0,48.5


In [18]:
lines.to_csv('{}lines_alternative.csv'.format(DATA_DIR))

### Historical rank

In [172]:
def extract_table(soup):
    '''in 2018, site puts spaces in the HTML.
    this strips out the spaces and extract the table.'''
    started = False
    table = ''
    for s in unicode(soup).split('\n'):
        if '<table' in s.replace(' ', ''):
            started = True
        if '</table>' in s.replace(' ', ''):
            table += s
            ## sometimes </table> then <table> are in the same line
            ## leave it open in this case
            if '<table' not in s.replace(' ', ''):
                started = False
            
        if started is True:
            table += s
    table = table.replace('  ','__').replace(' ','').replace('__',' ')
    while (table[:6] != '<table') & (table != ''):
        table = table[1:]    
    return BeautifulSoup(table)

def soup_to_pandas(tbl):
    '''takes an HTML table in BeautifulSoup form
    and returns a pandas dataframe'''
    cols = []
    team_rank = {}
    for i, tr in enumerate(tbl.findAll('tr')):
        team_rank[i] = {}
        for j, td in enumerate(tr.findAll('td')):
            if i == 0:
                cols.append(td.text.strip())
            else:
                team_rank[i][cols[j]] = td.text.strip()
        if (i == 0) & (not cols):
            cols = map(lambda x: x.text.strip(), tr.findAll('th'))
    df = DataFrame.from_dict(team_rank, orient='index')
    ## quit here if it's empty
    if df.shape[0] == 0:
        return df
    df_cols = df.columns
    df = df[df[df_cols[0]] != df_cols[0]]
    df['idx'] = np.arange(1, df.shape[0]+1)
    return df.set_index('idx')

def rank_week_17_url(yr):
    '''week 17 for rank has inconsistent
    URL convention, so the years are
    hard-coded'''
    week_17_map = {
        '2007': 'URL',
        '2008': 'URL',
        '2009': 'URL',
        '2010': 'URL',
        '2011': 'URL',
        '2012': 'URL',
        '2013': 'URL',
        '2014': 'URL',
        '2015': 'URL',
        '2016': 'URL',
        '2017': 'URL',
        '2018': 'URL',
    }
    for extra_yr in np.arange(2019,2100):
        week_17_map[extra_yr] = '/{0}/URL'.format(extra_yr)
    return week_17_map[str(yr)]
    
def scrape_rank_week(week, year):
    '''for a given week/year, scrapes offense,
    defense, special teams rankings by week.
    potentially useful for features'''
    N_TEAMS = 32
    dfs = []
    base_url = 'URL'
    
    if week == 17:
        url = base_url + rank_week_17_url(year)
    else:
        url = base_url + '/{}/URL'.format(year, week)
    soup = ScrapeTools().get_soup(url, driver)
    
    data_tables = soup.findAll('table', {'class': 'stats'})
    if not data_tables:
        #return soup
        data_tables = extract_table(soup).findAll('table', {'class': 'stats'})

    if len(data_tables) == 0:
        print 'NO DATA TABLES FOUND'
        return
    for data_table in data_tables:
        ## base HTML table
        if ( ('TOTALrank' in data_table.text) 
                   | ('TOTALVOA' in data_table.text) ) \
               & (('OFFENSErank' in data_table.text) | ('OFFENSEVOA' in data_table.text)) \
               & ('W-L' in data_table.text):
            
            df = soup_to_pandas(data_table)
            if df.shape[0] == N_TEAMS:
                dfs.append(df)
        
        ## some weeks have a past schedule table as well
        elif 'PASTSCHED' in data_table.text or 'PAST SCHED' in data_table.text:
            df = soup_to_pandas(data_table)
            if df.shape[0] == N_TEAMS:
                dfs.append(df)
        else:
            pass
    
    ## combine the DataFrames
    key = 'TEAM'
    for i, t in enumerate(dfs):
        if i == 0:
            df = t
        else:
            newcols = [key] + list(set(t.columns) - set(df.columns))
            df = df.merge(t[newcols], left_on=key, right_on=key) 
    return df

In [173]:
def clean_cols(df):
    '''clean up/standardize column
    names for merging weekly dataframes'''
    col_map = dict(map(
        lambda x: (x, x.replace('.','').strip()), 
        df.columns
    ))
    return df.rename(columns=col_map)

def merge_dfs(a, b):
    '''function passed into reduce call.
    merges the weekly dataframes. standardize columns.
    filling with None where necessary to line up columns.'''
    a = clean_cols(a)
    b = clean_cols(b)
    
    if sorted(a.columns) != sorted(b.columns):
        for a_not_b in set(a) - set(b):
            b[a_not_b] = None
        for b_not_a in set(b) - set(a):
            a[b_not_a] = None    
    return a.append(b[a.columns]).reset_index(drop=True)

In [167]:
driver = ScrapeTools().get_driver()

In [174]:
!mkdir -p rank

dfs = []
started = True
for year in np.arange(2018, 2006, -1):
    for week in np.arange(17,18): # no week 17 ratings
        if started is True:
            print year, week
            week_rank = scrape_rank_week(week, year)
            if week_rank is None:
                print 'FAIL', week, year
            else:
                week_rank['Year'] = year
                week_rank['Week'] = week
                week_rank.to_csv('{}/rank/rank_{:04d}_{:02d}.csv'.format(DATA_DIR, year, week))

2018 17
2017 17
2016 17
2015 17
2014 17
2013 17
2012 17
2011 17
2010 17
2009 17
2008 17
2007 17


#### Clean rank
* In Week 8, they switch from TOTAL DAVE to WEIGHTED rank (except weeks 8-10 2007)
 * TOTAL DAVE: NULL weeks 8-16
 * WEIGHTED rank: NULL weeks 1-7 (for 2007, it's weeks 1-10)
* LASTWEEK: NULL week 1

In [217]:
import json as JSON
rank_team_map = JSON.load(open('team_map_for_rank.json','r'))

for i, (k,v) in enumerate(rank_team_map.iteritems()):
    print '{} --> {}'.format(k, v)
    if i >= 5:
        break

MIN --> Vikings
MIA --> Dolphins
CAR --> Panthers
ATL --> Falcons
DET --> Lions
CIN --> Bengals


In [218]:
rank_dfs = map(
    lambda rank: read_csv('{}rank/{}'.format(DATA_DIR, rank)),
    sorted(filter(
        lambda x: x.startswith('rank_2'), 
        os.listdir('{}rank'.format(DATA_DIR))
    ))
)


allcols = []
for i in range(len(rank_dfs)):
    d = rank_dfs[i]
    d.columns = map(lambda x: str(x).strip(), d.columns)
    
    drop_cols = filter(lambda c: 'Unnamed' in c, d.columns)
    drop_cols += [
        'RANK', 'NON-ADJTOT VOA', '', '\xc2\xa0',
        '#', 'rank', 'RK', 
    ]
    
    for c in drop_cols:
        if c in d.columns:
            d.drop(c, axis=1, inplace=True)

    allcols.extend(d.columns)
    allcols = list(set(allcols))

    ## map team abbreviations to names
    ## make sure no null team names
    assert d[d['TEAM'].isnull()].shape[0] == 0
    d['TEAM'] = d['TEAM'].map(rank_team_map)
    assert d[d['TEAM'].isnull()].shape[0] == 0

In [219]:
rank_all = reduce(lambda x,y: x.append(y).reset_index(drop=True), rank_dfs)

In [220]:
rank_df = rank_all.loc[
        :, ['Week',
             'Year',
             '...']

    ].rename(
        columns={'Year':'season', 'Week':'week','TEAM':'name'}
    )
rank_df.shape
rank_df = rank_df[~rank_df['OFFENSErank'].str.contains('OFFENSE')]
rank_df.shape

## D or WEIGHTED rank
rank_df['D_OR_WTDrank'] = rank_df[['TOTAL D','WEIGHTEDrank']].apply(
    lambda x: filter(lambda y: str(y) != 'nan', x), axis=1
)
assert rank_df['DAVE_OR_WTDrank'].apply(
                lambda x: len(x)
            ).value_counts().index.max() == 1

rank_df['D_OR_WTDrank'] = rank_df['D_OR_WTDrank'].apply(lambda x: x[0])

## increment week_id by 1, since we want to join with the week after
## e.g. week 1 rankings come after week 1, so they're used for 
## week 2
rank_df['week'] = rank_df['week'].apply(lambda x: x+1)

rank_df.to_csv('{}/rank/rank_alltime.csv'.format(DATA_DIR))

(6496, 18)

(6496, 18)

#### Inspect NULL values

In [36]:
for c in rank_df.columns:
    nulls = rank_df[rank_df[c].isnull()].groupby(['season','week']).size()
    if nulls.shape[0] > 0:
        c
        nulls

'LASTWEEK'

season  week
2007    1       32
2008    1       32
2009    1       32
2010    1       32
2011    1       32
2012    1       32
2013    1       32
2014    1       32
2015    1       32
2016    1       32
2017    1       32
2018    1       32
dtype: int64

'TOTAL DAVE'

season  week
2007    8       32
        9       32
        10      32
        11      32
        12      32
        13      32
        14      32
        15      32
        16      32
2008    8       32
        9       32
        10      32
        11      32
        12      32
        13      32
        14      32
        15      32
        16      32
2009    8       32
        9       32
        10      32
        11      32
        12      32
        13      32
        14      32
        15      32
        16      32
2010    8       32
        9       32
        10      32
        11      32
        12      32
        13      32
        14      32
        15      32
        16      32
2011    8       32
        9       32
        10      32
        11      32
        12      32
        13      32
        14      32
        15      32
        16      32
2012    8       32
        9       32
        10      32
        11      32
        12      32
        13      32
        14      32

'WEIGHTEDrank'

...

season  week
2007    1       32
        2       32
        3       32
        16      32
2008    1       32
        2       32
        3       32
        15      32
        16      32
2009    1       32
        2       32
        15      32
        16      32
2010    1       32
        2       32
        3       32
2011    1       32
        2       32
        3       32
        16      32
2012    1       32
        2       32
        3       32
2013    1       32
        2       32
        3       32
2014    1       32
        2       32
        3       32
2015    1       32
        2       32
        3       32
2016    1       32
        2       32
        3       32
2017    1       32
        2       32
        3       32
2018    1       32
        2       32
        3       32
dtype: int64

'ESTIM.WINS'

season  week
2007    1       32
        2       32
        3       32
        16      32
2008    1       32
        2       32
        3       32
        16      32
2009    1       32
        2       32
2010    1       32
        2       32
        3       32
2011    1       32
        2       32
        3       32
2012    1       32
        2       32
        3       32
2013    1       32
        2       32
        3       32
2014    1       32
        2       32
        3       32
2015    1       32
        2       32
        3       32
2016    1       32
        2       32
        3       32
2017    1       32
        2       32
        3       32
2018    1       32
        2       32
        3       32
dtype: int64

'VAR'

season  week
2007    1       32
        2       32
        3       32
        16      32
2008    1       32
        2       32
        3       32
        16      32
2009    1       32
        2       32
2010    1       32
        2       32
        3       32
2011    1       32
        2       32
        3       32
2012    1       32
        2       32
        3       32
2013    1       32
        2       32
        3       32
2014    1       32
        2       32
        3       32
2015    1       32
        2       32
        3       32
2016    1       32
        2       32
        3       32
2017    1       32
        2       32
        3       32
2018    1       32
        2       32
        3       32
dtype: int64

In [1]:
from collections import Counter
counts = DataFrame.from_dict(
        Counter(reduce(lambda x,y: x+y, map(lambda z: z.columns.tolist(), rank_dfs))),
        orient='index'
    ).sort_values(by=0, ascending=False)
counts[0] = counts[0] / float(len(rank_dfs))

counts['name'] = counts.index
counts['name'] = counts['name'].apply(
    lambda x: x.replace(' ', '')
)

counts.groupby('name').sum()

In [38]:
years = {}
weeks = {}
week_years = {}
for d in rank_dfs:
    d.columns = [
            c.replace(' ','').replace('.','') 
            for c in d.columns
        ]
    yr, wk = d[['Year','Week']].values[0]
    if yr not in years.keys():
        years[yr] = d.columns.tolist()
    else:
        years[yr] += d.columns.tolist()
    
    if wk not in weeks.keys():
        weeks[wk] = d.columns.tolist()
    else:
        weeks[wk] += d.columns.tolist()
        
    week_years[(wk,yr)] = d.columns.tolist()    
    
counters = {}
n = 11.
for i, (k,v) in enumerate(weeks.iteritems()):
    curr = DataFrame.from_dict(Counter(v), orient='index').rename(columns={0:k})
    curr[k] = curr[k].apply(lambda x: (n-x)/n)
    if i == 0:
        combined_wk = curr
    else:
        combined_wk = combined_wk.merge(curr, left_index=True, right_index=True, how='outer').fillna(1.)
        
    counters = {}
n = 16.
for i, (k,v) in enumerate(years.iteritems()):
    curr = DataFrame.from_dict(Counter(v), orient='index').rename(columns={0:k})
    curr[k] = curr[k].apply(lambda x: (n-x)/n)
    if i == 0:
        combined_yr = curr
    else:
        combined_yr = combined_yr.merge(curr, left_index=True, right_index=True, how='outer').fillna(1.)
    
for i, (k,v) in enumerate(week_years.iteritems()):
    curr = DataFrame.from_dict(Counter(v), orient='index').rename(columns={0:k})
    if i == 0:
        combined_wkyr = curr
    else:
        combined_wkyr = combined_wkyr.merge(curr, left_index=True, right_index=True, how='outer').fillna(0)

combined_wkyr = combined_wkyr.T

#### Drop:
* RK, idx

In [39]:
## NULL weeks 8-16
col = 'LASTWEEK'
combined_wk.T[combined_wk.T[col] > 0].sort_index()[[col]]
combined_yr.T[combined_yr.T[col] > 0].sort_index()[[col]]

,LASTWEEK
1,1.0


,LASTWEEK
2007,0.0625
2008,0.0625
2009,0.0625
2010,0.0625
2011,0.0625
2012,0.0625
2013,0.0625
2014,0.0625
2015,0.0625
2016,0.0625


### Game Stats -- Primary Data Source
* Step 1: write all HTML to disk
* Step 2: extract individual tables and write them to separate directories

In [48]:
driver = ScrapeTools().get_driver()

In [49]:
all_games = {}

for year in range(2008,2009):
    print len(all_games.keys())
    for week in np.arange(16,17):
        url = 'URL'\
                .format(year, week)

        soup = ScrapeTools().get_soup(url, driver)
        all_games[(week, year)] = map(
            lambda x: x.findAll('a')[0].attrs['href'], 
            soup.findAll('td', {'class': 'gamelink'})
        )

0


In [50]:
all_games

{(16, 2008): []}

In [627]:
DataFrame.from_dict(all_games, orient='index').to_csv('{}/site_all_games_urls_1225.csv'.format(DATA_DIR))

### Week 16, 2008 webpage is missing : 
* see here: URL
* use the below URL to get all game URLs for this week only:
  * URL

In [188]:
tmp = read_csv('{}/site_all_games_urls_1225.csv'.format(DATA_DIR), index_col='Unnamed: 0')

In [189]:
all_urls = []
for _, row in tmp.iterrows():
    all_urls.extend(row.dropna().tolist())

In [190]:
len(set(all_urls))
len((all_urls))

3193

7209

In [191]:
all_urls = list(set(all_urls))

In [195]:
DataFrame.from_dict(
    Counter(map(lambda x: x.split('/')[-1][:8], all_urls)), orient='index'
).sort_values(by=0, ascending=False).head()

,0
20181230,16
20120101,16
20081228,16
20171231,16
20160103,16


#### games_stats directory is raw html unicode

In [30]:
!mkdir -p {DATA_DIR}games_stats_1225

In [196]:
from collections import Counter
tmp = DataFrame.from_dict(Counter(map(lambda x: x.split('.')[0][:-3], files)), orient='index')

tmp['idx'] = tmp.index
tmp[tmp['idx'].str.startswith('200812')]

,0,idx
200812080,1,200812080
200812150,1,200812150
200812040,1,200812040
200812070,14,200812070
200812140,14,200812140
200812110,1,200812110
200812280,16,200812280
200812010,1,200812010


In [197]:
driver = ScrapeTools().get_driver()
GAMES_STATS_DIR = 'games_stats_1225'

files = os.listdir('{}{}/'.format(DATA_DIR, GAMES_STATS_DIR))

# 3177
len(all_urls)
for i, url in enumerate(all_urls):
    if i % 100 == 0:
        print i
    game_id = url.split('/')[-1].split('.')[0]
    if '{}.txt'.format(game_id) in files:
        pass
    else:
        url = 'URL/{}'.format(url)
        soup = ScrapeTools().get_soup(url, driver)
        with open('{}{}/{}.txt'.format(DATA_DIR, GAMES_STATS_DIR, game_id), 'w') as w:
            w.write(unicode(soup))

3193

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


#### Extract Data From Stored HTML

In [198]:
def extract_scorebox_team_info(div, curr_game_metadata, i):
    '''for the team divs in scorebox, extract the team name/link, 
    city, score, and coach name/link'''
    team = div.findAll('a', {'itemprop':'name'})[0]

    team_link = team.attrs['href']
    team_full_name = team.text
    team_name = team_full_name.split(' ')[-1]
    team_city = ' '.join(team_full_name.split(' ')[:-1])
    team_score = div.findAll('div', {'class': 'score'})[0].text
    team_coach_raw = div.findAll('div', {'class': 'datapoint'})[0]\
                        .findAll('a')[0]
    team_coach_name = team_coach_raw.text
    team_coach_id = team_coach_raw.attrs['href']

    curr_game_metadata['team{}_link'.format(i)] = team_link
    curr_game_metadata['team{}_fullname'.format(i)] = team_full_name
    curr_game_metadata['team{}_name'.format(i)] = team_name
    curr_game_metadata['team{}_city'.format(i)] = team_city
    curr_game_metadata['team{}_score'.format(i)] = team_score
    curr_game_metadata['team{}_coach_raw'.format(i)] = team_coach_raw
    curr_game_metadata['team{}_coach_name'.format(i)] = team_coach_name
    curr_game_metadata['team{}_coach_id'.format(i)] = team_coach_id    
    
    return curr_game_metadata

In [199]:
def clean_game_meta(div, field):
    '''converts attendance to integer, and
    strips unnecessary headers from strings
    '''
    div = div.replace('Stadium: ','')\
             .replace('Start Time: ','')\
             .replace('Attendance: ','')\
             .replace('Time of Game: ','')
    
    if field == 'attendance':
        return int(div.replace(',',''))
    else:
        return div.strip()

In [200]:
def extract_game_metadata(scorebox):
    '''takes in the scorebox div. parses the 3
    divs: team0, team1, game data, and returns a dict
    entry for this game'''

    div_children = filter(
        lambda ch: str(type(ch)) != "<class 'bs4.element.NavigableString'>",
        scorebox.children
    )

    div_teams = filter(
            lambda x: x.attrs.get('class', []) == [], 
            div_children
        )
    div_game = filter(
            lambda x: x.attrs.get('class', []) == ['scorebox_meta'], 
            div_children
        )[0]

    ## TEAM INFO
    curr_game_metadata = OrderedDict()
    for i, div in enumerate(div_teams):
        curr_game_metadata = extract_scorebox_team_info(div, curr_game_metadata, i) 

    ## GAME INFO
    ## it looks like there is no indicator of the field
    ## e.g. date, start time. feels gross but hardcoding
    ## is the only solution I can think of
    meta_order = ['date','time','stadium','attendance','duration']

    for field, div in zip(meta_order, div_game.findAll('div')):
        curr_game_metadata[field] = clean_game_meta(div.text, field)
        if field == 'stadium':
            curr_game_metadata['stadium_link'] = div.findAll('a')[0].attrs['href']

    return curr_game_metadata

In [201]:
def text_or_link(td):
    '''if the element contains a hyperlink, return the
    raw string. otherwise, return only the text'''
    
    ## some a tags do not have href. hrefs is a list
    ## of href values. if empty, there is no href and
    ## return the text
    hrefs = map(lambda x: x.attrs.get('href', None), td.findAll('a'))
    hrefs = filter(lambda x: (x is not None) & ('#' not in str(x)), hrefs)
        
    if hrefs:
        return td
    else:
        return td.text

def get_game_tables(soup, game_id):
    '''for the given file (BeautifulSoup object),
    extract all data tables, and return a dictionary
    of data tables for the game.'''
    file_tables = {}
    for tbl_container in soup.findAll('div', {'class':'table_outer_container'}):
        tbl = tbl_container.findAll('tbody')[-1]

        table_dict = {}
        for i, tr in enumerate(tbl.findAll('tr')):
            if 'thead' in tr.attrs.get('class', []):
                pass
            else:
                table_dict[i] = OrderedDict()
                table_dict[i]['game_id'] = game_id

                ## first column is a th, but can be treated same as td
                for td in tr.findAll('th') + tr.findAll('td'):
                    if td.attrs['data-stat'] in ['player','name']:
                        name_type = td.attrs['data-stat']
                        table_dict[i][name_type+'_raw'] = text_or_link(td)
                        table_dict[i][name_type+'_name'] = td.text
                        name = map(
                            lambda x: x.attrs['href'], td.findAll('a')
                        )
                        if name:
                            table_dict[i][name_type+'_id'] = name[0]
                    else:
                        table_dict[i][td.attrs['data-stat']] = text_or_link(td)
                        
        table_name = tbl_container.findAll('caption')[0].text
        file_tables[table_name] = DataFrame.from_dict(table_dict, orient='index')
    return file_tables

In [202]:
def merge_split_tables(game_metadata, game_tables):
    '''certain data is broken out by team (drives, starters, snap counts). 
    this is indicated by the team name\' presence in the table key.
    make a Team column with the team name, and merge the DataFrames.
    '''

    team_names = map(lambda i: game_metadata['team{}_name'.format(i)], [0,1])
    tables_w_team_name = filter(
        lambda x: x.split(' ')[0] in team_names, 
        game_tables.keys()
    )

    for tbl in tables_w_team_name:
        assert 'Team' not in game_tables[tbl].columns
        game_tables[tbl]['Team'] = tbl.split(' ')[0]

        base_name = ' '.join(tbl.split(' ')[1:])

        if base_name not in game_tables.keys():
            game_tables[base_name] = game_tables[tbl]
        else:
            game_tables[base_name] = game_tables[base_name].append(
                game_tables[tbl]
            ).reset_index(drop=True)

    for tbl in tables_w_team_name:
        del game_tables[tbl]
        
    return game_tables

In [203]:
GAMES_STATS_DIR = '{}games_stats_1225'.format(DATA_DIR)

#### find net new files to process

In [211]:
# !rm {DATA_DIR}game_metadata.csv

files = sorted(os.listdir(GAMES_STATS_DIR))
len(files)
todo_ids = set(map(lambda x: x.split('.')[0], files))

existing_ids = set(
    read_csv('{}/game_metadata.csv'.format(DATA_DIR))
        ['game_id'].values.tolist()
)

new_files = map(lambda x: '{}.txt'.format(x), todo_ids - existing_ids)
len(new_files)

3193

32

In [2]:
start_pt = 0

for i, f in enumerate(new_files):
    if i % 100 == 0:
        print i

    if i >= start_pt:
        try:
            soup = BeautifulSoup(
                open('{}/{}'.format(GAMES_STATS_DIR, f)).read(), 
                'lxml'
            )

            scorebox = soup.findAll('div', {'class': 'scorebox'})[0]
            game_metadata = extract_game_metadata(scorebox)
                            
            game_tables = get_game_tables(soup, f.split('.')[0])
            game_tables = merge_split_tables(game_metadata, game_tables)

            for k,v in game_tables.iteritems():
                directory = k.replace(' ', '_')\
                             .replace('-','_')\
                             .replace('/','_')\
                             .replace('&','and')\
                             .replace(',','_')\
                             .lower()\
                             .replace('_table','')
                filename = f.replace('txt','csv')

                !mkdir -p {DATA_DIR}{directory}
                filepath = '{}/{}/{}'.format(DATA_DIR, directory, filename)
                if not os.path.exists(filepath):
                    v.to_csv(filepath)
    
            game_metadata_df = DataFrame.from_dict(
                    game_metadata, orient='index'
                ).T
            game_metadata_df['game_id'] = game_tables['Game Info Table'].loc[0, 'game_id']
            if os.path.exists('{}/game_metadata.csv'.format(DATA_DIR)):
                header = False
            else:
                header = True
            game_metadata_df.to_csv(
                '{}/game_metadata.csv'.format(DATA_DIR), 
                 mode='a', 
                 header=header,
                 index=False
                )
        except Exception as e:
            print 'FAILED: {}'.format(f)
            print e

### Stadiums
* #### Use wikipedia python package, scrape Wikipedia pages and extract lat/long for each stadium -- goal is to eventually compute travel distances between games
* #### Make API calls to geonames.org to get timezone (GMT offset)

In [10]:
def extract_lat_long(stadium_name):
    '''given a WikipediaPage object, use 
    BeautifulSoup to extract/return the
    lat/long for the stadium'''
    soup = BeautifulSoup(
        wikipedia.WikipediaPage(title = stadium_name).html()
    )
    geo = soup.findAll('span', {'class':'geo'})
    if geo:
        lat_long = map(float, geo[0].text.split('; '))
        return {'lat': lat_long[0], 'long': lat_long[1]}
    
def get_tz(user, lat, lon):
    '''pandas UDF: given geonames username,
    lat, and lon, make a REST API call
    and return the timezone and GMT offset'''
    base_url = 'http://api.geonames.org/timezone'
    base_url += '?lat={LAT}&lng={LONG}&username={USER}'
    xml = requests.get(
            base_url.format(LAT=lat, LONG=lon, USER=user)
        ).text
    
    return BeautifulSoup(xml)\
                .findAll('gmtoffset')\
                [0].text


In [7]:
stadium_name_fixes = {
    'TIAA Bank Stadium':'TIAA Bank Field'
}

stadiums = read_csv('{}/game_metadata.csv'.format(DATA_DIR))['stadium'].unique()
stadium_geo = {}
for s in stadiums:
    stadium_geo[s] = extract_lat_long(stadium_name_fixes.get(s, s))

In [20]:
stadium_df = DataFrame.from_dict(stadium_geo, orient='index')
stadium_df.index.name = 'stadium'
user = open('{}/.username'.format(DATA_DIR)).read().strip()
stadium_df['timezone'] = stadium_df[['lat','long']].apply(
    lambda x: get_tz(user, *x), axis=1
)
stadium_df.to_csv('{}/stadiums_w_tz.csv'.format(DATA_DIR))

### Get Annual Divisions

In [144]:
def extract_season_divisions(soup, divisions, season):
    '''given a BeautifulSoup object for a
    season standings page, get the team:division
    mapping for that season'''
    for conf in ['AFC','NFC']:
        table = soup.findAll('table', {'id':conf})[0].findAll('tbody')[0]
        for tr in table.findAll('tr'):
            ## division name change
            if len(tr.findAll('th')) == 0:
                division = str(tr.text.strip())
            else:
                teamname = str(
                    tr.findAll('th')[0].text.strip()
                      .replace('*','')
                      .replace('+','')
                    )
                divisions[(teamname, season)] = {
                    'division': division,
                }
    return divisions

In [145]:
driver = ScrapeTools().get_driver()

In [146]:
divisions = {}
for season in range(2007,2019):
    url = 'URL'\
            .format(season)

    soup = ScrapeTools().get_soup(url, driver)
    divisions = extract_season_divisions(soup, divisions, season)

In [153]:
divisions_df = DataFrame.from_dict(divisions, orient='index')\
                    .reset_index(drop=False)
divisions_df.columns = ['team','season','division']
divisions_df.index.name = 'idx'
divisions_df.to_csv('{}/team_divisions.csv'.format(DATA_DIR))

In [154]:
divisions_df['season'].value_counts()
divisions_df.head()

2018    32
2017    32
2016    32
2015    32
2014    32
2013    32
2012    32
2011    32
2010    32
2009    32
2008    32
2007    32
Name: season, dtype: int64

,team,season,division
idx,,,
0,Arizona Cardinals,2007,NFC West
1,Arizona Cardinals,2008,NFC West
2,Arizona Cardinals,2009,NFC West
3,Arizona Cardinals,2010,NFC West
4,Arizona Cardinals,2011,NFC West


### scrape the Season and Week # for each game ID
* get a list of teams
* for each year/team, scrape list of (game_id, season, week)
* NOTE: this has 2 entries for each game_id, one from each team's perspective. game_loc_team_ref indicates the perspective (and offensive team)

In [143]:
def extract_gamelog_from_table(tbody):
    '''given an HTML table, extract the gamelog elements.
    headers are taken from HTML tags rather than from table
    headers. called by extract_gamelog_from_soup'''
    data = {}
    for i, tr in enumerate(tbody.findAll('tr')):
        data[i] = {'week_id': tr.findAll('th', {'data-stat':'week_num'})[0].text}
        for j, td in enumerate(tr.findAll('td')):
            ## get game URL/ID
            if td.attrs.get('data-stat','') == 'boxscore_word':
                data[i]['game_id'] = td.findAll('a')[0]\
                                        .attrs['href']\
                                        .split('/')[-1].split('.')[0]
            else:
                colname = td.attrs['data-stat']
                data[i][colname] = td.text

    return DataFrame.from_dict(data, orient='index')

def extract_gamelog_from_soup(soup):
    tables = soup.findAll(
        'table', 
        {'class': 'row_summable sortable stats_table now_sortable sliding_cols'}
    )

    ## tables are duped
    if len(tables) == 4:
        tables = tables[:2]
    elif len(tables) == 2:
        tables = tables[:1]
    else:
        print '{}: too many tables'.format(team_season)

    dfs = [
        extract_gamelog_from_table(
                curr_tbl.findAll('tbody')[0]
            )
        for curr_tbl in tables
    ]

    return reduce(
        lambda df1,df2: df1.append(df2).reset_index(drop=True), 
        dfs
    )

In [153]:
games_df = read_csv('{}/game_metadata.csv'.format(DATA_DIR))
all_team_seasons = set(map(lambda x: x[0], games_df[['team0_link']].values)) \
                    | set(map(lambda x: x[0], games_df[['team1_link']].values))


driver = ScrapeTools().get_driver()
for i, team_season in enumerate(all_team_seasons):
    url = team_season.replace('.htm','/gamelog/')
    url = 'URL' + url
    soup = ScrapeTools().get_soup(url, driver)
    gamelog = extract_gamelog_from_soup(soup)
    ## add a column with the season
    gamelog['season'] = team_season.split('/')[-1].split('.')[0]
    gamelog['game_loc_team_ref'] =  team_season.split('/')[-2]
    
    if i == 0:
        all_gamelogs = gamelog
    else:
        all_gamelogs = all_gamelogs.append(gamelog).reset_index(drop=True)
        
    assert all_gamelogs.drop_duplicates().shape[0] == all_gamelogs.shape[0]

all_gamelogs.to_csv('{}/gamelogs.csv'.format(DATA_DIR), index=False)